## Pytorch

In [1]:
import torch
from det3d.torchie import Config
from det3d.models import build_detector
from det3d.torchie.trainer import load_checkpoint

config = "/workspace/centerformer/configs/nusc/nuscenes_centerformer_poolformer.py"

cfg = Config.fromfile(config)
checkpoint_path = "/workspace/centerformer/work_dirs/nuscenes_poolformer/poolformer.pth"

model = build_detector(cfg.model, train_cfg=None, test_cfg=cfg.test_cfg)
checkpoint = load_checkpoint(model, checkpoint_path, map_location="cpu")
model.cuda()
model.eval();

/data/centerformer/det3d/core/bbox/geometry.py:149: NumbaDeprecationWarning: The keyword argument 'nopython=False' was supplied. From Numba 0.59.0 the default is being changed to True and use of 'nopython=False' will raise a warning as the argument will have no effect. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit(nopython=False)
/data/centerformer/det3d/core/bbox/geometry.py:162: NumbaDeprecationWarning: The keyword argument 'nopython=False' was supplied. From Numba 0.59.0 the default is being changed to True and use of 'nopython=False' will raise a warning as the argument will have no effect. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit(nopython=False)
/data/centerformer/det3d/core/bbox/geometry.py:280: NumbaDeprecationWarning: The 'nopython' keyword argument was

Deformable Convolution not built!
Deformable Convolution not built!
[12/15/2023-08:18:23] [TRT] [W] CUDA lazy loading is not enabled. Enabling it can significantly reduce device memory usage. See `CUDA_MODULE_LOADING` in https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#env-vars
Use HM Bias:  -2.19


In [2]:
import pickle
pickle_dir = "/workspace/centerformer/work_dirs/partition/sample_data/"

with open(pickle_dir + "findcenter_input.pkl", 'rb') as handle:
    x = pickle.load(handle)
        

In [3]:
with torch.no_grad():
    ct_feat, center_pos_embedding, out_scores, out_labels, out_orders, out_masks = model.neck.find_centers(x)

## onnxruntime

In [4]:
import onnxruntime as ort
import numpy as np

onnx_path = "findCenter_folded.onnx"


with open(pickle_dir + "findcenter_input.pkl", 'rb') as handle:
    x = pickle.load(handle)
input_ort = x.cpu().numpy()
providers = ['CUDAExecutionProvider', 'CPUExecutionProvider']
ort_sess = ort.InferenceSession(onnx_path, providers=providers)
output_names = [output.name for output in ort_sess.get_outputs()]

In [5]:
ct_feat_ort, center_pos_embedding_ort, out_scores_ort, out_labels_ort, out_orders_ort, out_masks_ort = ort_sess.run(output_names, {'input_tensor': input_ort})

In [6]:
print(out_scores[0][0][:10]) #torch
print(out_scores_ort[0][0][:10]) #onnxruntime

tensor([0.8610, 0.8600, 0.8393, 0.8083, 0.8044, 0.7641, 0.7413, 0.7288, 0.7048,
        0.6847], device='cuda:0')
[0.86102206 0.8599623  0.83925366 0.80829763 0.8044028  0.764102
 0.7413236  0.72877914 0.704822   0.6847456 ]
